In [49]:
import os
os.environ["MKL_THREADING_LAYER"] = "TBB"

import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.neighbors import KNeighborsClassifier
import joblib
import matplotlib.pyplot as plt
import time

In [50]:
features_to_use = ['lbp', 'hog', 'color', 'gabor'] #Added all features for baseline.
use_pca = True #This way we can enable/disable PCA.
pca_variance_threshold = 0.95 #Keep components explaining 95% of variance.

models = {
    'Random Forest': RandomForestClassifier(random_state=1),
    'Gradient Boosting': GradientBoostingClassifier(random_state=1),
    'SVM': SVC(random_state=1),
    'KNN': KNeighborsClassifier(n_neighbors=1)
}

In [51]:
#These functions help set a loop/pipeline for experimenting with different features/models.
def load_and_combine_features(real_path, fake_path, features_to_use):
    #We load real and fake features, combine them with labels.
    real_data = np.load(real_path)
    fake_data = np.load(fake_path)

    #Print individual feature dimensions.
    print(f"  Feature dimensions in {real_path}:")
    for feature in features_to_use:
        print(f"    {feature}: {real_data[feature].shape[1]} features")
    
    #We extract and concatenate selected features.
    real_features = np.concatenate([real_data[feature] for feature in features_to_use], axis=1)
    fake_features = np.concatenate([fake_data[feature] for feature in features_to_use], axis=1)

    #We combine real and fake images.
    X = np.vstack([real_features, fake_features])
    y = np.hstack([np.zeros(len(real_features)), np.ones(len(fake_features))])
    return X, y

def print_metrics(y_true, y_pred, y_pred_proba, dataset_name, model_name):
    #We choose the following evaluation metrics.
    Acc = accuracy_score(y_true, y_pred)
    F1 = f1_score(y_true, y_pred)
    Precision = precision_score(y_true, y_pred)
    Recall = recall_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    
    #Calculate ROC-AUC.
    if y_pred_proba is not None:
        roc_auc = roc_auc_score(y_true, y_pred_proba)
    else:
        roc_auc = None
    
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    
    #Extract confusion matrix values.
    tn, fp, fn, tp = cm.ravel()

    print(f"\n{model_name} - {dataset_name} Results:")
    print(f" Accuracy: {Acc:.4f}")
    print(f" Precision: {Precision:.4f}")
    print(f" Recall: {Recall:.4f}")
    print(f" F1 Score: {F1:.4f}")
    if roc_auc is not None:
        print(f" ROC-AUC: {roc_auc:.4f}")
    print(f" Confusion Matrix:")
    print(f"   {cm}")

    print(f"\n CONFUSION MATRIX BREAKDOWN:")
    print(f"  True Negatives  (TN): {tn:4d} - Correctly identified as Fake")
    print(f"  False Positives (FP): {fp:4d} - Fake wrongly identified as Real")
    print(f"  False Negatives (FN): {fn:4d} - Real wrongly identified as Fake")
    print(f"  True Positives  (TP): {tp:4d} - Correctly identified as Real")
    
    #Detailed Error analysis.
    total_errors = fp + fn
    total_samples = len(y_true)
    print(f"\n ERROR ANALYSIS:")
    print(f"  Total Errors: {total_errors}/{total_samples} ({total_errors/total_samples*100:.2f}%)")
    print(f"  False Positives: {fp} ({fp/total_samples*100:.2f}%) - Fake images classified as Real")
    print(f"  False Negatives: {fn} ({fn/total_samples*100:.2f}%) - Real images classified as Fake")
    
    return roc_auc

In [52]:
#Time to load the data.
print("\n--- Train ---")
X_train, y_train = load_and_combine_features(
    'train_real_all_features.npz',
    'train_fake_all_features.npz',
    features_to_use
)
print("\n--- Validation ---")
X_valid, y_valid = load_and_combine_features(
    'valid_real_all_features.npz',
    'valid_fake_all_features.npz',
    features_to_use
)
print("\n--- Test ---")
X_test, y_test = load_and_combine_features(
    'test_real_all_features.npz',
    'test_fake_all_features.npz',
    features_to_use
)

print(f"\nTrain set: {X_train.shape}, Valid set: {X_valid.shape}, Test set: {X_test.shape}")
print(f"Total features used: {X_train.shape[1]}")

print("\n--- Class Balance ---")
print(f"Train - Real: {np.sum(y_train == 0)}, Fake: {np.sum(y_train == 1)}")
print(f"Valid - Real: {np.sum(y_valid == 0)}, Fake: {np.sum(y_valid == 1)}")
print(f"Test  - Real: {np.sum(y_test == 0)}, Fake: {np.sum(y_test == 1)}")


--- Train ---
  Feature dimensions in train_real_all_features.npz:
    lbp: 59 features
    hog: 1764 features
    color: 6 features
    gabor: 8 features

--- Validation ---
  Feature dimensions in valid_real_all_features.npz:
    lbp: 59 features
    hog: 1764 features
    color: 6 features
    gabor: 8 features

--- Test ---
  Feature dimensions in test_real_all_features.npz:
    lbp: 59 features
    hog: 1764 features
    color: 6 features
    gabor: 8 features

Train set: (10000, 1837), Valid set: (2000, 1837), Test set: (2000, 1837)
Total features used: 1837

--- Class Balance ---
Train - Real: 5000, Fake: 5000
Valid - Real: 1000, Fake: 1000
Test  - Real: 1000, Fake: 1000


In [53]:
#Scaling all features (before PCA).
print("\nScaling features")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

#Save scaler.
joblib.dump(scaler, 'scaler.joblib')
print("Scaler saved to 'scaler.joblib'")


Scaling features
Scaler saved to 'scaler.joblib'


In [54]:
from sklearn.decomposition import PCA
#Apply PCA.
if use_pca:
    #We fit PCA on training data only.
    pca = PCA(n_components=pca_variance_threshold, random_state=1)
    X_train_pca = pca.fit_transform(X_train_scaled)

    #We transform validation and test using the fitted PCA.
    X_valid_pca = pca.transform(X_valid_scaled)
    X_test_pca = pca.transform(X_test_scaled)

    n_components = pca.n_components_
    explained_var = np.sum(pca.explained_variance_ratio_)
    print(f"\nPCA Results:")
    print(f" Original features: {X_train_scaled.shape[1]}")
    print(f" Components retained: {n_components}")
    print(f" Explained variance: {explained_var:.4f} ({explained_var*100:.2f}%)")
    print(f" Dimensionality reduction: {X_train_scaled.shape[1]} --> {n_components}")

    joblib.dump(pca, 'pca.joblib')
    print("PCA saved to 'pca.joblib")

    #Use PCA-transformed data for training.
    X_train_scaled = X_train_pca
    X_valid_scaled = X_valid_pca
    X_test_scaled = X_test_pca

else:
    print("\nPCA Disabled")


PCA Results:
 Original features: 1837
 Components retained: 480
 Explained variance: 0.9501 (95.01%)
 Dimensionality reduction: 1837 --> 480
PCA saved to 'pca.joblib


In [55]:
import time 

#Time to train and evaluate all models.
trained_models = {}
roc_data = {} #Store ROC curve data for plotting.

for model_name, model in models.items():
    print(f"\n{'+'*25}")
    print(f"Training {model_name}")

    #Start timing.
    start_time = time.time()
    
    #Training.
    model.fit(X_train_scaled, y_train)
    
    #End timing.
    training_time = time.time() - start_time
    print(f"\nTraining completed in {training_time:.2f} seconds")

    #Predicting on all sets.
    y_train_pred = model.predict(X_train_scaled)
    y_valid_pred = model.predict(X_valid_scaled)
    y_test_pred = model.predict(X_test_scaled)
    
    # Check if model supports predict_proba to get probability predictions for ROC-AUC.
    if hasattr(model, "predict_proba"):
        y_train_proba = model.predict_proba(X_train_scaled)[:, 1]
        y_valid_proba = model.predict_proba(X_valid_scaled)[:, 1]
        y_test_proba = model.predict_proba(X_test_scaled)[:, 1]
    elif hasattr(model, "decision_function"):  #For SVM.
        y_train_proba = model.decision_function(X_train_scaled)
        y_valid_proba = model.decision_function(X_valid_scaled)
        y_test_proba = model.decision_function(X_test_scaled)
    else:
        y_train_proba = None
        y_valid_proba = None
        y_test_proba = None

    #Printing metrics.
    print_metrics(y_train, y_train_pred, y_train_proba, "Train", model_name)
    valid_auc = print_metrics(y_valid, y_valid_pred, y_valid_proba, "Valid", model_name)
    test_auc = print_metrics(y_test, y_test_pred, y_test_proba, "Test", model_name)
    
    #Store ROC curve data for validation set.
    if y_valid_proba is not None:
        fpr, tpr, _ = roc_curve(y_valid, y_valid_proba)
        roc_data[model_name] = {
            'fpr': fpr, 
            'tpr': tpr, 
            'auc': valid_auc,
            'training_time': training_time
        }

    #Saving models.
    model_filename = f"{model_name.lower().replace(' ', '_')}_model.joblib"
    joblib.dump(model, model_filename)
    print(f"\nModel saved to '{model_filename}'")

    #Storing in dictionary.
    trained_models[model_name] = model

print("\n" + "="*25)
print("Baseline: All models trained and saved with all features (lbp, hog, color, gabor)")


+++++++++++++++++++++++++
Training Random Forest

Training completed in 15.07 seconds

Random Forest - Train Results:
 Accuracy: 1.0000
 Precision: 1.0000
 Recall: 1.0000
 F1 Score: 1.0000
 ROC-AUC: 1.0000
 Confusion Matrix:
   [[5000    0]
 [   0 5000]]

 CONFUSION MATRIX BREAKDOWN:
  True Negatives  (TN): 5000 - Correctly identified as Fake
  False Positives (FP):    0 - Fake wrongly identified as Real
  False Negatives (FN):    0 - Real wrongly identified as Fake
  True Positives  (TP): 5000 - Correctly identified as Real

 ERROR ANALYSIS:
  Total Errors: 0/10000 (0.00%)
  False Positives: 0 (0.00%) - Fake images classified as Real
  False Negatives: 0 (0.00%) - Real images classified as Fake

Random Forest - Valid Results:
 Accuracy: 0.7195
 Precision: 0.7318
 Recall: 0.6930
 F1 Score: 0.7119
 ROC-AUC: 0.7912
 Confusion Matrix:
   [[746 254]
 [307 693]]

 CONFUSION MATRIX BREAKDOWN:
  True Negatives  (TN):  746 - Correctly identified as Fake
  False Positives (FP):  254 - Fake wro